In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
sys.path.append('/data/home/jdavey/notebooks/fastai/mi')
from data.loader import *
from nn.lin import *
from nn.relu import *
from nn.mse import *

In [4]:
import math
import torch

In [5]:
avaliable_urls()
x_train,y_train,x_valid,y_valid = get_data()

MNIST_URL http://deeplearning.net/data/mnist/mnist.pkl


In [6]:
def normalize(x, m, s): return (x-m)/s
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [8]:
# num hidden
nh = 50

In [9]:
# simplified kaiming init / he init
#784, 50
w1 = torch.randn(m,nh)*math.sqrt(2./m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)*math.sqrt(2./nh)
b2 = torch.zeros(1)

In [10]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [11]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [12]:
%time loss = model(x_train, y_train); loss

CPU times: user 254 ms, sys: 12.4 ms, total: 266 ms
Wall time: 186 ms


tensor(28.5752)

In [13]:
%time model.backward()

CPU times: user 230 ms, sys: 51.4 ms, total: 281 ms
Wall time: 60.4 ms


In [14]:
learning_rate = 0.0001
epochs = 10

In [15]:
for epoch in range(epochs):
    model.backward()
    parameters = [w1,b1,w2,b2]
    for parameter in parameters:
        parameter-=learning_rate*parameter.g
    loss = model(x_train, y_train)
    print(loss)

tensor(26.9713)
tensor(25.4717)
tensor(24.0717)
tensor(22.7680)
tensor(21.5569)
tensor(20.4356)
tensor(19.4002)
tensor(18.4452)
tensor(17.5652)
tensor(16.7550)
